**Tester le code avec des événements plus probable!!!**
## To do list:
* Q1-1
#####* Splitting (Diane) (faudrait jouer avec le nombre de niveaux de splitting, donc avec la valeur de la variable "seuil", pour voir ce qui donne la meilleure précision)
#####* Echantillonage d'importance (Peng-Wei) (C'est bon!!, à tester différent f)
#####* Chaîne de Markov
#####* Graphique : Comparaison de différentes méthodes.

* Q1-2
#####* Quantile processus AR(1) (comme dans calcul liste_a Q1-1) (Diane)
#####* Quantile échantillonnage d'importance (Diane)
* Q2
#####* M-C (Peng-Wei)

* Q3
# Finished :
* Q1
#####* Monte-Carlo naïve
* Q


In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt

## Q1

Dans cette première modélisation simplifiée, on considère donc que le prix $P_t$ est un processus de Poisson de paramètres $\lambda, \nu$ où $\nu$ est la loi des incréments $J_n$. 

Pour un temps d'attente moyen entre deux sauts de $300s$, on prend $\lambda = \dfrac{1}{300}$


In [3]:
# On identifie le processus par le processus de Poisson composé, qui finit à un temps fixé T
# On crée des fonctions pour modéliser le processus.

#### On définit nu, la loi des incréments
# Ancien : plus lent
saut_1_ancien = lambda x: np.random.choice([-1, 1], size=x, replace=True, p=[0.5, 0.5]) #correspond à m=1
saut_2_ancien = lambda x: np.random.choice([-3, -2, -1, 1, 2, 3], size=x, replace=True, p=0.5*np.array([1/6, 1/3, 1/2, 1/2, 1/3, 1/6])) #correspond à m=3

# Nouveau : beaucoup plus vite
value_1 = np.array([-1, 1])
value_2 = np.array([-3, -2, -2, -1, -1, -1, 1, 1, 1, 2, 2, 3])
saut_1 = lambda x : value_1[np.random.randint(low=2, size=x)]
saut_2 = lambda x : value_2[np.random.randint(low=12, size=x)]

# Les paramètres
P0 = 35
T = 4*60*60
lamb = 1/300

### Q2-2 Quantile

#### MCMC Naif

In [0]:
#Naif - Méthode 1


def Quantile_naif_2(seuil,n,lamb1,lamb2,T,P0,saut):

    liste_P_t = np.zeros(n)
    for i in range(n):
        N1 = np.random.poisson(lamb1*T)
        N2 = np.random.poisson(lamb2*T)
        P2 = 0 
        if (N2%2!=0): #si N2 est pair, la contribution du processus 2 vaut 0
            P2=-1+2*np.random.binomial(1,0.5)
        increments = saut(N1)
        
        liste_P_t[i] = np.sum(increments)+P0 +P2
    liste_P_t = np.sort(liste_P_t)
    return liste_P_t[int(np.ceil(n*seuil))-1]

#### MCMC numba

In [4]:
#MC Accéléré - Méthode 2 (pas plus rapide ?)

# On identifie le processus par le processus de Poisson composé, qui finit à un temps fixé T
# On crée des fonctions pour modéliser le processus.


def Q2_2_MC(P0, T, lamb1, lamb2, saut):
  # saut : une fonction qui prend la taille de sortie comme entrée pour les sauts.
  N1 = np.random.poisson(lam=lamb1 * T)
  T1 = np.sort(np.random.uniform(low=0.0, high=T, size=N1))
  J1 = saut(N1)

  N2 = np.random.poisson(lam=lamb2 * T)
  T2 = np.sort(np.random.uniform(low=0.0, high=T, size=N2))

  J2_0 = saut_1(1)[0]

  somme = P0
  index_T2 = 0
  for i in range(len(T1)):
    while index_T2 < N2 and T2[index_T2] < T1[i]:
      somme += J2_0
      J2_0 *= -1
      index_T2 += 1
    somme += J1[i]
  return somme                      # 1 échantillon

def Q2_2_MC_n(P0, T, lamb1, lamb2, saut, size):
  res = []
  for i in range(size):
    res.append(Q2_2_MC(P0, T, lamb1, lamb2, saut))
  return np.array(res)

def quantile(echantillion, size, quant):
  index_bot = int(math.ceil(size * quant))
  index_top = int(math.ceil(size * (1 - quant)))
  echantillion.partition((index_bot, index_top))
  return echantillion[index_bot], echantillion[index_top]

def Question2_2_naif(size, quant, saut, P0, T, lamb1, lamb2):
  return quantile(Q2_2_MC_n(P0, T, lamb1, lamb2, saut, size), size, quant)

In [5]:
%%time 
P0 = 35
T = 4*60*60
lamb1 = 1/660
lamb2 = 1/110
n = int(1e8)
quantiles = Q2_2_MC_n(P0, T, lamb1, lamb2, saut_1, n)

print(quantiles[99], quantiles[999], quantiles[9999], quantiles[99990000], quantiles[99999000], quantiles[99999900])

#(19, 52) pour m=1 seuil 10^-4 n =10^4
#(14,56) pour m=1 seuil 10^-5 n =10^5
#Donne ... pour n = 10^6 et seuil=1e-6 et m=1
#Donne (0, 69) pour n = 10^4 et seuil=1e-4 et m=3
#Donne (-2, 71) pour n = 10^5 et seuil=1e-5 et m=3
#Donne pour n = 10^5 et seuil=1e-6 et m=3

KeyboardInterrupt: 

In [ ]:
%%time
quantiles = Q2_2_MC_n(P0, T, lamb1, lamb2, saut_2, n)

print(quantiles[99], quantiles[999], quantiles[9999], quantiles[99990000], quantiles[99999000], quantiles[99999900])
#Donne 17/55 pour n = 10^4 et seuil=1e-4 et m=1
#Donne (12.0, 56.0) pour n = 10^5 et seuil=1e-5 et m=1
#Donne (11.0, 58.0) pour n = 10^6 et seuil=1e-6 et m=1
#Donne (2.0, 65.0) pour n = 10^4 et seuil=1e-4 et m=3
#Donne (-4.0, 73.0) pour n = 10^5 et seuil=1e-5 et m=3
#Donne (-10.0, 77.0) pour n = 10^6 et seuil=1e-6 et m=3